In [32]:
import sys
import json
import pandas as pd
import requests
from datetime import datetime
from bson import ObjectId
from pymongo import MongoClient

In [3]:
df = pd.read_csv('tripadvisor_merged.csv')

In [4]:
museum_info = df[['MuseumName', 'Address','Description','Fee','LengthOfVisit','PhoneNum','Rating','ReviewCount']]

In [5]:
museum_info.insert(0, 'museum_id', range(1, len(museum_info) + 1))

In [28]:
museum_info.head()

,MuseumName,Address,Description,Fee,Langtitude,Latitude,LengthOfVisit,PhoneNum,Rating,ReviewCount,TotalThingsToDo
0,Newseum,"555 Pennsylvania Ave NW, Washington DC, DC 200...",Find out for yourself why everyone is calling ...,Yes,-77.019235,38.893138,2-3 hours,+1 888-639-7386,4.5,6309.0,398.0
1,The Metropolitan Museum of Art,"1000 5th Ave, New York City, NY 10028-0198",At New York City's most visited museum and att...,Yes,-73.962928,40.779165,2-3 hours,1 212-535-7710,5.0,36627.0,1028.0
2,The National WWII Museum,"945 Magazine Street, New Orleans, LA 70130-3813","Founded by historian and author, Stephen Ambro...",NaN,-90.070086,29.943004,NaN,+1 504-528-1944,5.0,15611.0,319.0
3,Denver Museum of Nature & Science,"2001 N Colorado Blvd, Denver, CO 80205-5798",The Denver Museum of Nature & Science is the R...,NaN,-104.941020,39.769189,NaN,303-370-6000,4.5,2564.0,231.0
4,Art Institute of Chicago,"111 S Michigan Ave, Chicago, IL 60603-6488","This Classical Renaissance structure, guarded ...",Yes,-87.623724,41.879547,More than 3 hours,312 443 3600,5.0,15532.0,614.0


In [6]:
import re

def format_phone_number(phone):
    if not isinstance(phone, str):
        return None  # or you can return a placeholder string if you want
    # Remove any character that is not a digit
    digits = re.sub(r'\D', '', phone)
    # Check if the phone number has 10 digits (or 11 digits if it includes country code '1')
    if len(digits) == 10 or (len(digits) == 11 and digits.startswith('1')):
        formatted_phone = '+1 ' + '-'.join([digits[-10:-7], digits[-7:-4], digits[-4:]])
        return formatted_phone
    else:
        return '+1 ' + phone  # Keeping original format if it doesn't match expected patterns

# Apply the function to the 'PhoneNum' column
museum_info['PhoneNum'] = museum_info['PhoneNum'].apply(format_phone_number)


/var/folders/25/28bgdb_97bz6shswlftbfy_00000gn/T/ipykernel_11066/3903347352.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_info['PhoneNum'] = museum_info['PhoneNum'].apply(format_phone_number)


In [7]:
museum_info['PhoneNum'] = museum_info['PhoneNum'].apply(format_phone_number)

/var/folders/25/28bgdb_97bz6shswlftbfy_00000gn/T/ipykernel_11066/2068209846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_info['PhoneNum'] = museum_info['PhoneNum'].apply(format_phone_number)


In [12]:
museum_info.Rating = 0
museum_info.ReviewCount = 0

/var/folders/25/28bgdb_97bz6shswlftbfy_00000gn/T/ipykernel_11066/249981736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_info.Rating = 0
/var/folders/25/28bgdb_97bz6shswlftbfy_00000gn/T/ipykernel_11066/249981736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  museum_info.ReviewCount = 0


In [13]:
museum_info.head()

,museum_id,MuseumName,Address,Description,Fee,LengthOfVisit,PhoneNum,Rating,ReviewCount
0,1,Newseum,"555 Pennsylvania Ave NW, Washington DC, DC 200...",Find out for yourself why everyone is calling ...,Yes,2-3 hours,+1 888-639-7386,0,0
1,2,The Metropolitan Museum of Art,"1000 5th Ave, New York City, NY 10028-0198",At New York City's most visited museum and att...,Yes,2-3 hours,+1 212-535-7710,0,0
2,3,The National WWII Museum,"945 Magazine Street, New Orleans, LA 70130-3813","Founded by historian and author, Stephen Ambro...",NaN,NaN,+1 504-528-1944,0,0
3,4,Denver Museum of Nature & Science,"2001 N Colorado Blvd, Denver, CO 80205-5798",The Denver Museum of Nature & Science is the R...,NaN,NaN,+1 303-370-6000,0,0
4,5,Art Institute of Chicago,"111 S Michigan Ave, Chicago, IL 60603-6488","This Classical Renaissance structure, guarded ...",Yes,More than 3 hours,+1 312-443-3600,0,0


In [15]:
museum_info.to_csv('museum_info.csv', index=False)

In [3]:
reviews_info = pd.read_csv('Review Data.csv')

In [24]:
reviews_info

,museum_id,MuseumName,ReviewText,Rating
0,1,Newseum,Good,4
1,1,Newseum,Bad,1
2,1,Newseum,okay,3
3,1,Newseum,Haha,2
4,1,Newseum,will come again,5
5,2,The Metropolitan Museum of Art,not that good,2
6,2,The Metropolitan Museum of Art,I like it,1
7,2,The Metropolitan Museum of Art,won't come again,4
8,2,The Metropolitan Museum of Art,nevermind,3
9,2,The Metropolitan Museum of Art,we love it,4


In [4]:
museum_info = pd.read_csv('museum_info.csv')

In [5]:
def hash_museum_name(museum_name):
    ascii_sum = sum(ord(char) for char in museum_name)
    return ascii_sum % 3

In [15]:
db_uris = [
    'mongodb://ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017/database1',
    'mongodb://ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017/database2',
    'mongodb://ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017/database3'
]

In [16]:
clients = [MongoClient(uri) for uri in db_uris]
dbs = [client.get_default_database() for client in clients]

In [17]:
# Distribute and insert data
for _, row in museum_info.iterrows():
    hash_value = hash_museum_name(row['MuseumName'])
    # Select the appropriate database based on the hash value
    db = dbs[hash_value]
    # Insert the document (you may want to convert the row to a dictionary)
    db.museum_collection.insert_one(row.to_dict())

In [ ]:
## adding reviews

In [37]:
def submit_review(museum_name, rating, review, dbs, hash_museum_name):
    # Find the target database based on the hash value of the museum name
    db_index = hash_museum_name(museum_name)
    target_db = dbs[db_index]

    # Here you would use target_db to interact with the database
    # Since we cannot perform actual database operations, please use this in your local setup
    print(f"Database selected: {target_db}")
    print(f"Submitting review for {museum_name}: '{review}' with rating {rating}")

    # Assuming the museum document has a 'museum_name' field that is unique
    museum_document = target_db['museum_collection'].find_one({'MuseumName': museum_name})
    if museum_document:
        # Update the average rating and review count
        new_review_count = museum_document['ReviewCount'] + 1
        new_avg_rating = ((museum_document['Rating'] * museum_document['ReviewCount']) + rating) / new_review_count

        # Update the museum document in the museum_collection
        target_db['museum_collection'].update_one(
            {'MuseumName': museum_name},
            {
                '$set': {'Rating': new_avg_rating},
                '$inc': {'ReviewCount': 1}
            }
        )

        # Insert the new review into the review_collection
        review_document = {
            'MuseumName': museum_name,
            'review': review,
            'rating': rating,
            'timestamp': datetime.now()
        }
        target_db['review_collection'].insert_one(review_document)


In [38]:
## samples
reviews = [
    ("Newseum", 5, "Amazing experience, very interactive!"),
    ("Newseum", 4, "Great exhibits, but quite crowded."),
    ("Art Institute of Chicago", 3, "so so"),
    ("Art Institute of Chicago", 5, "AMAZING!")
]

for museum_name, rating, review_text in reviews:
    submit_review(museum_name, rating, review_text, dbs, hash_museum_name)


Database selected: Database(MongoClient(host=['ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017'], document_class=dict, tz_aware=False, connect=True), 'database3')
Submitting review for Newseum: 'Amazing experience, very interactive!' with rating 5
Database selected: Database(MongoClient(host=['ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017'], document_class=dict, tz_aware=False, connect=True), 'database3')
Submitting review for Newseum: 'Great exhibits, but quite crowded.' with rating 4
Database selected: Database(MongoClient(host=['ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017'], document_class=dict, tz_aware=False, connect=True), 'database1')
Submitting review for Art Institute of Chicago: 'so so' with rating 3
Database selected: Database(MongoClient(host=['ec2-13-48-25-237.eu-north-1.compute.amazonaws.com:27017'], document_class=dict, tz_aware=False, connect=True), 'database1')
Submitting review for Art Institute of Chicago: 'AMAZING!' with rating 5


In [ ]:
## deleting review

In [39]:
from pymongo import MongoClient
from bson import ObjectId

def delete_review(review_id, dbs):
    # Find the review and its associated museum_id
    for db in dbs:  # Assuming dbs is a list of your database connections
        review = db.review_collection.find_one({'_id': ObjectId(review_id)})
        if review:
            museum_id = review['museum_id']
            break
    
    if not review:
        print("Review not found in any database.")
        return

    # Now that you have the museum_id, you can proceed to delete the review
    db.review_collection.delete_one({'_id': ObjectId(review_id)})

    # Recalculate the average rating and review count for the museum
    reviews_cursor = db.review_collection.find({'museum_id': museum_id})
    total_rating, review_count = 0, 0
    for review in reviews_cursor:
        total_rating += review['rating']
        review_count += 1
    new_avg_rating = total_rating / review_count if review_count else 0

    # Update the museum_collection
    db.museum_collection.update_one(
        {'_id': museum_id},
        {'$set': {'Rating': new_avg_rating, 'ReviewCount': review_count}}
    )

In [ ]:
# Example usage
review_id = 'review_object_id_here'
delete_review(review_id, dbs)